# Finding routes between schools and health facilities  
The CIESIN GRID3 data provides high-resolution geospatial data that can be used to analyze various aspects of infrastructure and services in Nigeria. One practical application of this data is to find optimal routes between schools and health facilities. Below is an example of how to achieve this using Python and relevant libraries.

In [ ]:
# install neccessary packages
!pip install osmnx
!pip install mapclassify
!pip install folium
!pip install momepy

In [ ]:
# import
import geopandas as gpd
import networkx as nx
import momepy
import osmnx as ox
import matplotlib.pyplot as plt
from itertools import product
import numpy as np
import mapclassify
import folium
import json
import pandas as pd
import momepy 
import requests 


In [ ]:
# install arcgis API for accessing data
# check that arcgis is installed 
!pip install arcgis
    

In [ ]:
# import arcgis
from arcgis.gis import GIS 


In [ ]:
# Anonymously authenticate arcgis API
gis = GIS()

In [ ]:
def fetch_nga_roads(bbox, crs, record_limit, url):
    """
    Fetch NGA roads from an ArcGIS FeatureServer within a bounding box.

    Parameters
    ----------
    bbox : dict
        Bounding box with keys xmin, ymin, xmax, ymax (in EPSG:3857).
    crs : str, optional
        CRS for the output GeoDataFrame (default: EPSG:3857).
    record_limit : int, optional
        Maximum number of records per request (default: 200000).
    url : str, optional
        ArcGIS FeatureServer query URL.

    Returns
    -------
    geopandas.GeoDataFrame
        GeoDataFrame containing all fetched road features.
    """

    params = {
        "f": "geojson",
        "where": "1=1",
        "geometry": json.dumps({
            **bbox,
            "spatialReference": {"wkid": int(crs.split(":")[1])},
        }),
        "geometryType": "esriGeometryEnvelope",
        "spatialRel": "esriSpatialRelContains",
        "outFields": "*",
        "returnGeometry": "true",
        "resultRecordCount": record_limit,
    }

    all_features = []
    offset = 0

    while True:
        paged_params = params | {"resultOffset": offset}
        r = requests.get(url, params=paged_params)
        r.raise_for_status()

        data = r.json()
        features = data.get("features", [])

        if not features:
            break

        all_features.extend(features)
        offset += len(features)
        
    valid_features = [
        f for f in all_features
        if f.get("geometry") is not None]

    return gpd.GeoDataFrame.from_features(valid_features, crs=crs)

In [ ]:
# Read in service for NGA roads
url = "https://services3.arcgis.com/BU6Aadhn6tbBEdyk/arcgis/rest/services/GRID3_NGA_roads/FeatureServer/0/query"

boundbox =  {
        "xmin": 360102.4944,
        "ymin": 725614.4996,
        "xmax": 381199.1042,
        "ymax": 746837.3928  }



gdf = fetch_nga_roads(boundbox, crs="EPSG:3857", record_limit=1000, url=url)

gdf 

In [ ]:
# Plot roads
gdf.plot()

In [ ]:
exploded_road_gdf = gdf.copy()

exploded_road_gdf = exploded_road_gdf.set_crs(epsg=4326, allow_override=True)
    
exploded_road_gdf = exploded_road_gdf.to_crs('EPSG:3857')

exploded_road_gdf = exploded_road_gdf.explode(index_parts=True) 


exploded_road_gdf['length_meters'] = exploded_road_gdf.geometry.length 

# Build network
Gs = momepy.gdf_to_nx(
    exploded_road_gdf,
    directed='MultiDiGraph',           
    approach='primal',
    integer_labels=True,
    preserve_index=True,
    length='length_meters'        
)


# 🔧 OSMnx compatibility: alias length_meters → length
for _, _, data in Gs.edges(data=True):
    data["length"] = data["length_meters"] 




# # Back to GeoDataFrames
nodes, edges = momepy.nx_to_gdf(Gs, points=True, lines=True)
 

# # Try roundabout simplification
edges = momepy.roundabout_simplification(edges)
 
 
edges

In [ ]:

# check simplification group
edges['simplification_group'].value_counts()

In [ ]:
# Plot nodes and edges
ax = edges.plot(figsize=(10,10), color="gray", lw=1.0)
ax = nodes.plot(ax=ax, color="green", markersize=4)

In [ ]:
# Calculate time to traverse a road segment in minutes using the speed estimate and length
edges["travel_time_mins"] = ((edges["length_meters"]/1000) / (edges["speed_estimate"])) * 60

# drop if travel_time_mins is infinite or NaN or less than equal to zero
edges = edges.replace([np.inf, -np.inf], np.nan)
edges = edges.dropna(subset=['travel_time_mins'])
edges = edges[edges['travel_time_mins'] > 0]    

edges.head()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5))

# Plot speed estimate
ax1 = edges.plot(column="speed_estimate", legend=True,ax=ax1, cmap='viridis')
ax1.set_title('Speed (Km/h)')

# Plot travel time
ax2 = edges.plot(column="travel_time_mins", legend=True,ax=ax2, cmap='inferno',scheme='natural_breaks')
ax2.set_title('Travel Time (Minutes)')

plt.tight_layout()
plt.show()

In [ ]:
from pyproj import Transformer

# Geocode an origin and destination address
# Origin
orig_address = "Oshitelu St, Lagos, Nigeria"

orig_y, orig_x = ox.geocode(orig_address)  # notice the coordinate order (y, x)!

# Destination
dest_address = "Obafemi Awolowo Wy, Lagos, Nigeria"
dest_y, dest_x = ox.geocode(dest_address) 


# Transformer: lon/lat → graph CRS
transformer = Transformer.from_crs(
    "EPSG:4326",
    Gs.graph["crs"],
    always_xy=True
)

# Print coordinates
print("Origin coords:", orig_x, orig_y)
print("Destination coords:", dest_x, dest_y)

orig_x_p, orig_y_p = transformer.transform(orig_x, orig_y)
dest_x_p, dest_y_p = transformer.transform(dest_x, dest_y)

# Snap to graph
orig_node_id, dist_to_orig = ox.distance.nearest_nodes(
    Gs, X=orig_x_p, Y=orig_y_p, return_dist=True
)
dest_node_id, dist_to_dest = ox.distance.nearest_nodes(
    Gs, X=dest_x_p, Y=dest_y_p, return_dist=True
)

# Print distance to nearest node
print("Origin node-id:", orig_node_id, "and distance:", dist_to_orig, "meters.")
print("Destination node-id:", dest_node_id, "and distance:", dist_to_dest, "meters.")




# Calculate shortest path between origin and destionation nodes and the length
path = nx.shortest_path(Gs, source=orig_node_id, target=dest_node_id, weight='travel_time_mins')
length = nx.shortest_path_length(Gs, source=orig_node_id, target=dest_node_id, weight='travel_time_mins')





In [ ]:
length

In [ ]:
# Shortest path based on distance
fig, ax = ox.plot_graph_route(Gs, path)

# Add the travel time as title
ax.set_xlabel("Shortest path is {t: .1f} minutes.".format(t=length))

In [ ]:
# Explore graph edges and route together in a single map
route_edges = ox.routing.route_to_gdf(Gs, path, weight='length')

m = edges.explore(color="black", tiles="cartodbdarkmatter")
m = route_edges.explore(m=m, color="maroon", style_kwds={"weight": 5})

# Add the Esri World Imagery tile layer
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite',
    overlay=False,
    control=True
).add_to(m)

# Add a layer control to toggle between tile layers
folium.LayerControl().add_to(m)
m

In [ ]:
# Read in Nigeria health facilities layer
layer_url = "https://services3.arcgis.com/BU6Aadhn6tbBEdyk/arcgis/rest/services/GRID3_NGA_health_facilities_v2_0/FeatureServer/0/query"
 

# Create bounding box
query_extent = {
    "xmin": 3.328171,
    "ymin": 6.582383,
    "xmax": 3.383102,
    "ymax": 6.633909}



health_fac = fetch_nga_roads(query_extent, crs="EPSG:4326", record_limit=200000, url=layer_url)

 

health_fac.head(5)

In [ ]:
# Read in Nigeria schools layer
layer_url = "https://services3.arcgis.com/BU6Aadhn6tbBEdyk/arcgis/rest/services/Schools_in_Nigeria/FeatureServer/0/query"
# feature_layer = FeatureLayer(layer_url)

# Create bounding box
query_extent = {
    "xmin": 3.328171,
    "ymin": 6.582383,
    "xmax": 3.383102,
    "ymax": 6.633909,
     
}
 
schools = fetch_nga_roads(query_extent, crs="EPSG:4326", record_limit=200000, url=layer_url)

schools.head(5)

In [ ]:
#set same crs as graph
schools = schools.to_crs(Gs.graph["crs"])
health_fac = health_fac.to_crs(Gs.graph["crs"])

schools["longitude"] = schools.geometry.x
schools["latitude"] = schools.geometry.y

health_fac["longitude"] = health_fac.geometry.x
health_fac["latitude"] = health_fac.geometry.y


# Caclulate shortest route between every school and health facility using the travel_time_mins variable as the weight
# this can take a few minutes

weight_attribute = 'travel_time_mins'

# Find nearest nodes to each school and health facility
orig_node_id, dist_to_orig = ox.distance.nearest_nodes(
    Gs, 
    X=schools.geometry.x, Y=schools.geometry.y, return_dist=True)
dest_node_id, dist_to_dest = ox.distance.nearest_nodes(Gs, X=health_fac.geometry.x, Y=health_fac.geometry.y, return_dist=True)

shortest_paths = {}

for source in orig_node_id:
    for target in dest_node_id:
        try:
            # Get the actual path
            path = nx.shortest_path(Gs, source=source, target=target, weight=weight_attribute)
            # Get the path length
            length = nx.shortest_path_length(Gs, source=source, target=target, weight=weight_attribute)
            shortest_paths[(source, target)] = {'path': path, 'length': length}
        except nx.NetworkXNoPath:
          # build dictionary with shortest routes
            shortest_paths[(source, target)] = {'path': None, 'length': float('inf')}

print(shortest_paths)

print("Unique school nodes:", len(set(orig_node_id)))
print("Unique facility nodes:", len(set(dest_node_id)))
print("Snap distances (schools):", dist_to_orig.min(), dist_to_orig.max())
print("Snap distances (facilities):", dist_to_dest.min(), dist_to_dest.max())



In [ ]:
# create dataframe of shortest paths between every school and health facility and drop school/health facility combinations that aren't the closest
import pandas as pd
# build dataframe
df = pd.DataFrame(shortest_paths).T
df = df.reset_index()
df = df.rename(columns={'level_0': 'school_node', 'level_1': 'health_facility_node'})
df = df.replace([np.inf, -np.inf], np.nan)

# sort by length and keep only the shortest routes for each school
df = df.sort_values(by=['school_node', 'length'], ascending=[True, True])
df = df.drop_duplicates(subset=['school_node'], keep='first')
df

In [ ]:
# merge nearest node ID back to schools and health facitility geodataframes

# set node ids
schools['node_id'] = orig_node_id
health_fac['node_id'] = dest_node_id

# get only columns needed for merge
school_merge = schools[['node_id', 'name','latitude','longitude']]
health_merge = health_fac[['node_id', 'facility_name']]

# merge nearest routes with school and health facility information
nearest_df = df.merge(school_merge, left_on='school_node', right_on='node_id')
nearest_df = nearest_df.merge(health_merge, left_on='health_facility_node', right_on='node_id')

# drop duplicates
nearest_df1 = nearest_df.drop_duplicates(subset=['school_node', 'length'])
nearest_df1

In [ ]:
# visualize all nearest routes
import random
import matplotlib.colors as mcolors
# drop routes with nans
nearest_df1 = nearest_df1.dropna(subset=['path'])

# convert to list
routes = nearest_df1['path'].tolist()

# set route colors
num_paths = len(routes)
color_pool = list(mcolors.CSS4_COLORS.values()) # 148+ colors
path_colors = random.choices(color_pool, k=num_paths)

# plot map

transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

xmin, ymin = transformer.transform(3.328171, 6.582383)
xmax, ymax = transformer.transform(3.383102, 6.633909)

bbox_3857 = (xmin, ymin, xmax, ymax)
    
fig, ax = ox.plot_graph_routes(Gs, routes,route_colors=path_colors, route_linewidth=6, node_size=0, bbox=bbox_3857)

plt.show()

In [ ]:
# Explore a route interactively

# select school to view
nearest_df1_s = nearest_df1[nearest_df1['name'] == 'Saint Joseph Secondary School']

route_edges = ox.routing.route_to_gdf(Gs, nearest_df1_s.iloc[0].path, weight='length')

# map facilities, roads, and routes
m = edges.explore(color="black", tiles="cartodbdarkmatter")
m = route_edges.explore(m=m, color="yellow", style_kwds={"weight": 5})
m = schools.explore(m=m, color = "blue", style_kwds={"weight": 5})
m = health_fac.explore(m=m, color = "red", style_kwds={"weight": 5})
# add the Esri World Imagery tile layer to interactive map
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite',
    overlay=False,
    control=True
).add_to(m)

# add a layer control to toggle between tile layers
folium.LayerControl().add_to(m)
m

In [ ]:
# Map the travel time from each school to the nearset health facility - darker dots are closer to a health facility, red dots are health facilities

# construct geodataframe
geometry = gpd.points_from_xy(nearest_df1['longitude'], nearest_df1['latitude'])
nearest_gdf = gpd.GeoDataFrame(nearest_df1, geometry=geometry, crs='EPSG:4326')

edges_plot = edges.to_crs(epsg=4326)
health_plot = health_fac.to_crs(epsg=4326)


# Plot with length column for travel time
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
edges.plot(ax=ax, color='grey', alpha=.1, zorder=1)
health_fac.plot(marker='+', color='red', legend=True, ax=ax, markersize=5, zorder=3)
nearest_gdf.plot(column='length', cmap='inferno', legend=True, ax=ax, markersize=25, zorder=2)
plt.title("Time from school to nearest health facility")
ax.set_xlim(bbox_3857[0], bbox_3857[2])
ax.set_ylim(bbox_3857[1], bbox_3857[3]  )
plt.show()